In [2]:
import json
import os
import pickle
import subprocess
import sys
import wget

In [8]:
# find bzip file from "https://dumps.wikimedia.org/enwiki
wiki_url = "https://dumps.wikimedia.org/enwiki/20190501/enwiki-20190501-pages-articles-multistream.xml.bz2"
output_dir = "text"
target_dir = "wiki"
output_file = "enwiki.txt"

In [4]:
# download full text
if os.path.basename(wiki_url) not in os.listdir():
    wget.download(wiki_url)

In [5]:
if "wikiextractor" not in os.listdir():
    subprocess.call("git clone https://github.com/attardi/wikiextractor.git".split())

In [6]:
if output_dir not in os.listdir():
    subprocess.call(["python", "wikiextractor/WikiExtractor.py", os.path.basename(wiki_url),
                    "-o", output_dir, "--json"])

In [7]:
def remove_some_char(text):
    title = text.replace("'", "").lower()
    for i in ".:?/*\\":
        title = title.replace(i, "")
    for i in ["CON", "PRN", "AUX", "NUL", " "]:
        if title == i:
            title += "-reserved"
    title = title.replace(" ", "_")
    return title

In [17]:
if target_dir not in os.listdir():
    os.mkdir(target_dir)

minimum_length = 500

title_dict = {}
error_num = 0
for path in os.listdir(output_dir):
    original_path = os.path.join(output_dir, path)
    target_path = os.path.join(target_dir, path)
    if path not in os.listdir(target_dir):
        os.mkdir(target_path)
        
    for file_path in os.listdir(original_path):        
        with open(os.path.join(original_path, file_path), "r", encoding="utf8") as fr:
            jsons = [json.loads(line) for line in fr]
            
        with open(os.path.join(target_path, file_path), "w", encoding="utf8") as fw:
            for js in jsons:
                if len(js["text"]) >= minimum_length:
                    fw.write(js["text"]+"\n")

# ubuntu
!cat wiki/\*/\* > enwiki.txt

# windows

In [21]:
with open(output_file, "w", encoding="utf8") as fw:
    for path in os.listdir(target_dir):
        target_path = os.path.join(target_dir, path)
        for file_path in os.listdir(target_path):
            with open(os.path.join(target_path, file_path), "r", encoding="utf8") as fr:
                [fw.write(line) for line in fr]